In [ ]:
%matplotlib inline
import pandas as pd
import pandas_profiling as pp
import numpy as np
import os
from sklearn.metrics.pairwise import haversine_distances
from math import radians

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
df_train = pd.read_csv("cs5228-2021-semester-2-final-project/train.csv")
df_train

,listing_id,name,street,type,model,market_segment,type_of_area,bedrooms,bathrooms,district,region,planning_area,subszone,lat,lng,tenure,built_year,no_of_units,area_size,eco_category,accessibility,date_listed,price
0,6998418,seascape,57 cove way,condominium,condominium,ocr,strata,3,4.0,4,central region,southern islands,sentosa,1.239337,103.837487,leasehold/99 years,2011.0,151.0,2336.0,uncategorized,guarded,2021-05-04,5390000.0
1,2046604,la maison,10 moulmein rise,apartment,apartment,ocr,strata,3,3.0,11,central region,novena,moulmein,1.319533,103.847030,freehold,1999.0,24.0,1259.0,uncategorized,guarded,2021-05-07,2310000.0
2,7563513,viva,2 suffolk walk,condominium,condominium,ocr,strata,4,3.0,11,central region,novena,moulmein,1.315656,103.844445,freehold,2012.0,235.0,1959.0,uncategorized,guarded,2021-06-30,5279500.0
3,3071445,urban treasures,205 jalan eunos,condominium,condominium,ocr,strata,3,2.0,14,east region,bedok,kaki bukit,1.329367,103.905791,freehold,NaN,237.0,883.0,uncategorized,guarded,2022-01-02,1843600.0
4,9667539,infini at east coast,east coast road,apartment,apartment,ocr,strata,3,3.0,15,east region,bedok,frankel,1.309176,103.911352,freehold,NaN,36.0,1066.0,uncategorized,guarded,2021-12-24,2262700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26043,2322717,citylights,jellicoe road,condominium,condominium,ocr,strata,3,3.0,8,central region,kallang,lavender,1.308896,103.863069,leasehold/99 years,2007.0,600.0,1442.0,uncategorized,guarded,2021-12-14,2750000.0
26044,3161844,dalvey haus,101 dalvey road,condominium,condominium,ocr,strata,3,3.0,10,central region,tanglin,nassim,1.316263,103.825190,freehold,NaN,17.0,1561.0,uncategorized,guarded,2022-01-05,6449300.0
26045,2402528,my manhattan,25 simei street 3,condominium,condominium,ocr,strata,2,2.0,18,east region,tampines,simei,1.341303,103.953348,leasehold/99 years,2014.0,301.0,883.0,uncategorized,guarded,2021-06-23,1306800.0
26046,8150354,j gateway,gateway drive,condominium,condominium,ocr,strata,4,3.0,22,west region,jurong east,jurong gateway,1.335557,103.742417,leasehold/99 years,2016.0,738.0,1206.0,uncategorized,guarded,2021-10-23,2420000.0


In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df_train)
profile.to_file(output_file="train_profile.html")

In [ ]:
df_test = pd.read_csv("cs5228-2021-semester-2-final-project/test.csv")
df_test

,listing_id,name,street,type,model,market_segment,type_of_area,bedrooms,bathrooms,district,region,planning_area,subszone,lat,lng,tenure,built_year,no_of_units,area_size,eco_category,accessibility,date_listed
0,1487111,leedon green,leedon heights,condominium,condominium,ocr,strata,2,2.0,10,central region,bukit timah,farrer court,1.313566,103.803218,freehold,NaN,638.0,710.0,uncategorized,guarded,2021-12-12
1,6794066,the line @ tanjong rhu,tanjong rhu road,condominium,condominium,ocr,strata,3,2.0,15,central region,kallang,tanjong rhu,1.298437,103.884408,freehold,2016.0,130.0,1055.0,uncategorized,guarded,2021-10-12
2,4027017,parc elegance,telok kurau road,apartment,apartment,ocr,strata,NaN,1.0,15,east region,bedok,frankel,1.317851,103.908905,freehold,2013.0,157.0,463.0,uncategorized,guarded,2021-12-23
3,1119141,the jovell,13 flora drive,condominium,condominium,ocr,strata,2,1.0,17,east region,pasir ris,flora drive,1.358005,103.965725,leasehold/99 years,1930.0,428.0,645.0,uncategorized,guarded,2021-12-31
4,7374262,shenton way,5 shenton way,condominium,condominium,ocr,strata,4,6.0,1,central region,downtown core,cecil,1.277474,103.849419,leasehold/99 years,NaN,NaN,6200.0,uncategorized,guarded,2021-07-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,8295867,river place,havelock road,condominium,condominium,ocr,strata,1,1.0,3,central region,singapore river,robertson quay,1.289610,103.841276,leasehold/99 years,1999.0,492.0,743.0,uncategorized,guarded,2021-12-20
7496,9827623,costa rhu,rhu cross,condominium,condominium,ocr,strata,2,2.0,15,central region,kallang,tanjong rhu,1.296610,103.867069,leasehold/99 years,1997.0,737.0,1345.0,uncategorized,guarded,2021-11-28
7497,7319135,phoenix residences,phoenix avenue,apartment,apartment,ocr,strata,3+1,NaN,23,west region,bukit batok,gombak,1.375255,103.757531,leasehold/99 years,NaN,74.0,1033.0,uncategorized,guarded,2022-01-01
7498,1321491,parc botannia,fernvale street,condominium,condominium,ocr,strata,2+1,1.0,28,north-east region,sengkang,fernvale,1.398588,103.874632,leasehold/99 years,2021.0,735.0,581.0,uncategorized,guarded,2021-12-28


In [ ]:
profile = ProfileReport(df_test)
profile.to_file(output_file="test_profile.html")

In [ ]:
aux_dir = 'cs5228-2021-semester-2-final-project/auxiliary-data/auxiliary-data/'
aux_csv = [f for f in os.listdir(aux_dir) if os.path.isfile(os.path.join(aux_dir,f))]

array([[    0.        , 11099.54035582, 15867.05630309],
       [11099.54035582,     0.        , 10721.56215443],
       [15867.05630309, 10721.56215443,     0.        ]])

In [ ]:
result[0]

array([    0.        , 11099.54035582, 11099.54035582])

In [ ]:
df_train[["lat", "lng"]]

,lat,lng
0,1.239337,103.837487
1,1.319533,103.847030
2,1.315656,103.844445
3,1.329367,103.905791
4,1.309176,103.911352
...,...,...
26043,1.308896,103.863069
26044,1.316263,103.825190
26045,1.341303,103.953348
26046,1.335557,103.742417


In [ ]:
aux_csv

['sg-commerical-centres.csv',
 'sg-gov-markets-hawker-centres.csv',
 'sg-primary-schools.csv',
 'sg-secondary-schools.csv',
 'sg-shopping-malls.csv',
 'sg-train-stations.csv']

In [ ]:
aux_train_stations = pd.read_csv(aux_dir+aux_csv[-1])
aux_mrt_stations = aux_train_stations[aux_train_stations["type"] == "mrt"]
aux_lrt_stations = aux_train_stations[aux_train_stations["type"] != "mrt"]
aux_mrt_stations

,name,codes,lat,lng,opening_year,type
0,Eunos,EW7,1.319778,103.903252,1989.0,mrt
1,Chinese Garden,EW25,1.342352,103.732596,1988.0,mrt
2,Khatib,NS14,1.417383,103.832980,1988.0,mrt
3,Kranji,NS7,1.425177,103.762165,1996.0,mrt
4,Redhill,EW18,1.289562,103.816816,1988.0,mrt
5,Yew Tee,NS5,1.397534,103.747405,1996.0,mrt
6,Pioneer,EW28,1.337586,103.697321,2009.0,mrt
7,Commonwealth,EW20,1.302438,103.798304,1988.0,mrt
8,Paya Lebar,EW8/CC9,1.317771,103.892619,1989.0,mrt
9,Simei,EW3,1.343202,103.953371,1989.0,mrt


In [ ]:
aux_lrt_stations.columns

Index(['name', 'codes', 'lat', 'lng', 'opening_year', 'type'], dtype='object')

In [ ]:
aux_train_stations["lat_radian"] = aux_train_stations.apply(lambda x: radians(x["lat"]), axis=1)
aux_train_stations["lng_radian"] = aux_train_stations.apply(lambda x: radians(x["lng"]), axis=1)
aux_train_stations

,name,codes,lat,lng,opening_year,type,lat_radian,lng_radian
0,Eunos,EW7,1.319778,103.903252,1989.0,mrt,0.023034,1.813454
1,Chinese Garden,EW25,1.342352,103.732596,1988.0,mrt,0.023428,1.810475
2,Khatib,NS14,1.417383,103.832980,1988.0,mrt,0.024738,1.812227
3,Kranji,NS7,1.425177,103.762165,1996.0,mrt,0.024874,1.810991
4,Redhill,EW18,1.289562,103.816816,1988.0,mrt,0.022507,1.811945
5,Yew Tee,NS5,1.397534,103.747405,1996.0,mrt,0.024392,1.810734
6,Pioneer,EW28,1.337586,103.697321,2009.0,mrt,0.023345,1.809860
7,Commonwealth,EW20,1.302438,103.798304,1988.0,mrt,0.022732,1.811622
8,Paya Lebar,EW8/CC9,1.317771,103.892619,1989.0,mrt,0.022999,1.813268
9,Simei,EW3,1.343202,103.953371,1989.0,mrt,0.023443,1.814329


In [ ]:

bsas = [-34.83333, -58.5166646]
paris = [49.0083899664, 2.53844117956]
coords = list(zip(bsas,paris))
bsas_in_radians = [radians(_) for _ in bsas]
paris_in_radians = [radians(_) for _ in paris]
list_new=coords+ (0.023034476467601664, 1.8134538532569344)
result = haversine_distances([bsas_in_radians, paris_in_radians,])
result = result * 6371000/1000  # multiply by Earth radius to get kilometers
result 

In [ ]:
distances_1 = []
distances_2 = []
distances_3 = []

coords = list(zip(aux_train_stations["lat_radian"], aux_train_stations["lng_radian"]))
k = 3
for idx, row in df_train.iterrows():
    location1 = [(radians(row["lat"]), radians(row["lng"]))]
    list_coords = location1 + coords
    distances = haversine_distances(list_coords)[0][1: ]
    distances *= 6371000/1000
    distances = sorted(distances)[ :k] 
    distances_1.append(distances[0])
    distances_2.append(distances[1])
    distances_3.append(distances[2])

df_train["train_dist1"] = distances_1
df_train["train_dist2"] = distances_2
df_train["train_dist3"] = distances_3
df_train

26048


,listing_id,name,street,type,model,market_segment,type_of_area,bedrooms,bathrooms,district,region,planning_area,subszone,lat,lng,tenure,built_year,no_of_units,area_size,eco_category,accessibility,date_listed,price,train_dist1,train_dist2,train_dist3
0,6998418,seascape,57 cove way,condominium,condominium,ocr,strata,3,4.0,4,central region,southern islands,sentosa,1.239337,103.837487,leasehold/99 years,2011.0,151.0,2336.0,uncategorized,guarded,2021-05-04,5390000.0,3.409821,4.238236,4.539417
1,2046604,la maison,10 moulmein rise,apartment,apartment,ocr,strata,3,3.0,11,central region,novena,moulmein,1.319533,103.847030,freehold,1999.0,24.0,1259.0,uncategorized,guarded,2021-05-07,2310000.0,0.370228,1.125888,1.250745
2,7563513,viva,2 suffolk walk,condominium,condominium,ocr,strata,4,3.0,11,central region,novena,moulmein,1.315656,103.844445,freehold,2012.0,235.0,1959.0,uncategorized,guarded,2021-06-30,5279500.0,0.536424,0.786985,1.093760
3,3071445,urban treasures,205 jalan eunos,condominium,condominium,ocr,strata,3,2.0,14,east region,bedok,kaki bukit,1.329367,103.905791,freehold,NaN,237.0,883.0,uncategorized,guarded,2022-01-02,1843600.0,0.689663,0.729867,1.102969
4,9667539,infini at east coast,east coast road,apartment,apartment,ocr,strata,3,3.0,15,east region,bedok,frankel,1.309176,103.911352,freehold,NaN,36.0,1066.0,uncategorized,guarded,2021-12-24,2262700.0,1.330838,1.483444,2.291250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26043,2322717,citylights,jellicoe road,condominium,condominium,ocr,strata,3,3.0,8,central region,kallang,lavender,1.308896,103.863069,leasehold/99 years,2007.0,600.0,1442.0,uncategorized,guarded,2021-12-14,2750000.0,0.173237,0.531141,0.929039
26044,3161844,dalvey haus,101 dalvey road,condominium,condominium,ocr,strata,3,3.0,10,central region,tanglin,nassim,1.316263,103.825190,freehold,NaN,17.0,1561.0,uncategorized,guarded,2022-01-05,6449300.0,0.432839,1.261879,1.459587
26045,2402528,my manhattan,25 simei street 3,condominium,condominium,ocr,strata,2,2.0,18,east region,tampines,simei,1.341303,103.953348,leasehold/99 years,2014.0,301.0,883.0,uncategorized,guarded,2021-06-23,1306800.0,0.211154,0.904354,1.188957
26046,8150354,j gateway,gateway drive,condominium,condominium,ocr,strata,4,3.0,22,west region,jurong east,jurong gateway,1.335557,103.742417,leasehold/99 years,2016.0,738.0,1206.0,uncategorized,guarded,2021-10-23,2420000.0,0.267858,1.327640,1.696205


In [ ]:
def aux_augmenting(df, aux_df, k, key_word):
    '''
    df (pd.DataFrame): parent DataFrame
    aux_df (pd.DataFrame): auxillary DataFrame
    k (int): number of distances to be taken
    key_word (str): type of auxillary data used for naming columns
    
    Returns:
    pd.DataFrame which contains the auxillary information 
    (i.e., closest k locations from every property of sale) from aux_df
    '''
    temp_dict = dict()
    for i in range(k):
        temp_dict[key_word + "_dist_" + str(i)] = list()
    
    aux_df["lat_radian"] = aux_df.apply(lambda x: radians(x["lat"]), axis=1)
    aux_df["lng_radian"] = aux_df.apply(lambda x: radians(x["lng"]), axis=1)
    coords = list(zip(aux_df["lat_radian"], aux_df["lng_radian"]))
        
    for idx, row in df.iterrows():
        location1 = [(radians(row["lat"]), radians(row["lng"]))]
        list_coords = location1 + coords
        
        distances = haversine_distances(list_coords)[0][1: ]
        distances *= 6371000/1000
        distances = sorted(distances)[ :k] 
        
        for i in range(k):
            temp_dict[key_word + "_dist_" + str(i)].append(distances[i])
    
    for key in temp_dict:
        df[key] = temp_dict[key]
    
    return df

In [ ]:
aux_csv

['sg-commerical-centres.csv',
 'sg-gov-markets-hawker-centres.csv',
 'sg-primary-schools.csv',
 'sg-secondary-schools.csv',
 'sg-shopping-malls.csv',
 'sg-train-stations.csv']

In [ ]:
df_train = pd.read_csv("cs5228-2021-semester-2-final-project/train.csv")

for csv_file in aux_csv:
    aux_df = pd.read_csv(aux_dir + csv_file)
    print(csv_file)
    df_train = aux_augmenting(df=df_train, aux_df=aux_df, k=3, key_word=csv_file.split(".")[0])

df_train

sg-commerical-centres.csv
sg-gov-markets-hawker-centres.csv
sg-primary-schools.csv
sg-secondary-schools.csv
sg-shopping-malls.csv
sg-train-stations.csv


,listing_id,name,street,type,model,market_segment,type_of_area,bedrooms,bathrooms,district,region,planning_area,subszone,lat,lng,tenure,built_year,no_of_units,area_size,eco_category,accessibility,date_listed,price,sg-commerical-centres_dist_0,sg-commerical-centres_dist_1,sg-commerical-centres_dist_2,sg-gov-markets-hawker-centres_dist_0,sg-gov-markets-hawker-centres_dist_1,sg-gov-markets-hawker-centres_dist_2,sg-primary-schools_dist_0,sg-primary-schools_dist_1,sg-primary-schools_dist_2,sg-secondary-schools_dist_0,sg-secondary-schools_dist_1,sg-secondary-schools_dist_2,sg-shopping-malls_dist_0,sg-shopping-malls_dist_1,sg-shopping-malls_dist_2,sg-train-stations_dist_0,sg-train-stations_dist_1,sg-train-stations_dist_2
0,6998418,seascape,57 cove way,condominium,condominium,ocr,strata,3,4.0,4,central region,southern islands,sentosa,1.239337,103.837487,leasehold/99 years,2011.0,151.0,2336.0,uncategorized,guarded,2021-05-04,5390000.0,5.604095,6.467607,6.497485,4.079855,4.201203,4.548944,4.064895,4.065760,4.243224,4.429845,5.219602,5.782498,3.218556,3.366116,3.990321,3.409821,4.238236,4.539417
1,2046604,la maison,10 moulmein rise,apartment,apartment,ocr,strata,3,3.0,11,central region,novena,moulmein,1.319533,103.847030,freehold,1999.0,24.0,1259.0,uncategorized,guarded,2021-05-07,2310000.0,0.407610,1.457892,2.634634,0.519457,0.903970,0.976741,0.235428,0.742936,0.897731,1.263926,1.533175,1.594012,0.339015,0.457519,0.457519,0.370228,1.125888,1.250745
2,7563513,viva,2 suffolk walk,condominium,condominium,ocr,strata,4,3.0,11,central region,novena,moulmein,1.315656,103.844445,freehold,2012.0,235.0,1959.0,uncategorized,guarded,2021-06-30,5279500.0,0.498692,1.929303,2.273753,0.648320,0.654110,1.259672,0.294793,0.790091,0.811455,1.075413,1.900441,1.911872,0.201332,0.201332,0.554531,0.536424,0.786985,1.093760
3,3071445,urban treasures,205 jalan eunos,condominium,condominium,ocr,strata,3,2.0,14,east region,bedok,kaki bukit,1.329367,103.905791,freehold,NaN,237.0,883.0,uncategorized,guarded,2022-01-02,1843600.0,1.848952,3.136191,4.448253,1.018186,1.024112,1.671550,0.483431,0.543315,0.638243,0.495752,1.708314,2.510505,1.684204,1.888515,2.038405,0.689663,0.729867,1.102969
4,9667539,infini at east coast,east coast road,apartment,apartment,ocr,strata,3,3.0,15,east region,bedok,frankel,1.309176,103.911352,freehold,NaN,36.0,1066.0,uncategorized,guarded,2021-12-24,2262700.0,2.221947,5.455112,6.171672,0.618976,0.940260,1.059401,0.376927,0.472848,0.837449,0.624355,0.718272,1.560606,0.822088,0.881869,0.940480,1.330838,1.483444,2.291250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26043,2322717,citylights,jellicoe road,condominium,condominium,ocr,strata,3,3.0,8,central region,kallang,lavender,1.308896,103.863069,leasehold/99 years,2007.0,600.0,1442.0,uncategorized,guarded,2021-12-14,2750000.0,2.010765,2.512385,2.637252,0.350262,0.675587,0.712781,0.852651,1.396924,1.506625,1.757201,1.903568,2.406505,0.124759,0.767367,0.805083,0.173237,0.531141,0.929039
26044,3161844,dalvey haus,101 dalvey road,condominium,condominium,ocr,strata,3,3.0,10,central region,tanglin,nassim,1.316263,103.825190,freehold,NaN,17.0,1561.0,uncategorized,guarded,2022-01-05,6449300.0,2.066410,3.138920,3.584091,1.506035,1.677513,2.168026,0.613752,1.133454,1.952133,0.613752,0.948020,1.225572,1.086251,1.213996,1.280818,0.432839,1.261879,1.459587
26045,2402528,my manhattan,25 simei street 3,condominium,condominium,ocr,strata,2,2.0,18,east region,tampines,simei,1.341303,103.953348,leasehold/99 years,2014.0,301.0,883.0,uncategorized,guarded,2021-06-23,1306800.0,1.135732,1.478839,1.602144,1.083730,1.915024,1.952622,0.203218,1.062367,1.141238,0.098837,1.369711,1.410914,0.319463,1.312741,1.328221,0.211154,0.904354,1.188957
26046,8150354,j gateway,gateway drive,condominium,condominium,ocr,strata,4,3.0,22,west region,jurong east,jurong gateway,1.335557,103.742417,leasehold/99 years

In [ ]:
df_test = pd.read_csv("cs5228-2021-semester-2-final-project/test.csv")

for csv_file in aux_csv:
    aux_df = pd.read_csv(aux_dir + csv_file)
    print(csv_file)
    df_test = aux_augmenting(df=df_test, aux_df=aux_df, k=3, key_word=csv_file.split(".")[0])

df_test

sg-commerical-centres.csv
sg-gov-markets-hawker-centres.csv
sg-primary-schools.csv
sg-secondary-schools.csv
sg-shopping-malls.csv
sg-train-stations.csv


,listing_id,name,street,type,model,market_segment,type_of_area,bedrooms,bathrooms,district,region,planning_area,subszone,lat,lng,tenure,built_year,no_of_units,area_size,eco_category,accessibility,date_listed,sg-commerical-centres_dist_0,sg-commerical-centres_dist_1,sg-commerical-centres_dist_2,sg-gov-markets-hawker-centres_dist_0,sg-gov-markets-hawker-centres_dist_1,sg-gov-markets-hawker-centres_dist_2,sg-primary-schools_dist_0,sg-primary-schools_dist_1,sg-primary-schools_dist_2,sg-secondary-schools_dist_0,sg-secondary-schools_dist_1,sg-secondary-schools_dist_2,sg-shopping-malls_dist_0,sg-shopping-malls_dist_1,sg-shopping-malls_dist_2,sg-train-stations_dist_0,sg-train-stations_dist_1,sg-train-stations_dist_2
0,1487111,leedon green,leedon heights,condominium,condominium,ocr,strata,2,2.0,10,central region,bukit timah,farrer court,1.313566,103.803218,freehold,NaN,638.0,710.0,uncategorized,guarded,2021-12-12,1.659712,2.282740,2.859211,0.410398,0.799763,0.959639,0.980559,1.577959,1.872602,1.048619,1.239575,1.414103,1.128913,1.806787,1.898364,0.654270,0.804495,1.352592
1,6794066,the line @ tanjong rhu,tanjong rhu road,condominium,condominium,ocr,strata,3,2.0,15,central region,kallang,tanjong rhu,1.298437,103.884408,freehold,2016.0,130.0,1055.0,uncategorized,guarded,2021-10-12,2.415213,3.566028,3.798724,0.446516,0.965068,1.105996,1.469982,1.939220,2.135100,0.249159,1.019529,1.123224,0.969806,1.409195,1.959446,0.888254,1.119644,1.202684
2,4027017,parc elegance,telok kurau road,apartment,apartment,ocr,strata,NaN,1.0,15,east region,bedok,frankel,1.317851,103.908905,freehold,2013.0,157.0,463.0,uncategorized,guarded,2021-12-23,1.712861,4.455317,5.509744,0.587265,1.224805,1.282656,0.893951,0.970922,1.042381,1.343460,1.357290,1.593958,1.475109,1.531509,1.565824,0.572349,0.663977,1.721414
3,1119141,the jovell,13 flora drive,condominium,condominium,ocr,strata,2,1.0,17,east region,pasir ris,flora drive,1.358005,103.965725,leasehold/99 years,1930.0,428.0,645.0,uncategorized,guarded,2021-12-31,1.862085,2.345213,2.545142,2.334125,2.724174,2.888535,0.709727,0.967805,1.601576,0.528898,1.089132,1.135445,2.319693,2.371475,2.392085,1.249331,1.869678,2.143628
4,7374262,shenton way,5 shenton way,condominium,condominium,ocr,strata,4,6.0,1,central region,downtown core,cecil,1.277474,103.849419,leasehold/99 years,NaN,NaN,6200.0,uncategorized,guarded,2021-07-14,1.179363,2.070123,4.777679,0.327226,0.459676,0.648850,1.075392,2.359387,2.395381,1.680849,2.403544,3.023519,0.394375,0.583026,0.630811,0.409125,0.439039,0.549620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,8295867,river place,havelock road,condominium,condominium,ocr,strata,1,1.0,3,central region,singapore river,robertson quay,1.289610,103.841276,leasehold/99 years,1999.0,492.0,743.0,uncategorized,guarded,2021-12-20,1.235432,1.506703,3.389243,0.202035,0.546793,0.765924,0.754614,1.541732,1.722011,0.591775,1.361700,1.958200,0.430967,0.523329,0.590674,0.465637,0.602379,0.635354
7496,9827623,costa rhu,rhu cross,condominium,condominium,ocr,strata,2,2.0,15,central region,kallang,tanjong rhu,1.296610,103.867069,leasehold/99 years,1997.0,737.0,1345.0,uncategorized,guarded,2021-11-28,1.772288,1.863059,3.701360,0.781770,1.087292,1.522150,1.351177,2.503936,2.511494,1.697411,2.023579,2.501422,0.920440,0.945451,0.996750,0.518447,0.789131,1.152102
7497,7319135,phoenix residences,phoenix avenue,apartment,apartment,ocr,strata,3+1,NaN,23,west region,bukit batok,gombak,1.375255,103.757531,leasehold/99 years,NaN,74.0,1033.0,uncategorized,guarded,2022-01-01,4.557207,5.244413,5.383403,1.690009,4.169747,4.401163,0.808886,0.957476,1.050726,0.720402,0.908465,1.133790,0.639621,0.738360,0.833287,0.378166,0.453321,0.609647
7498,1321491,parc botannia,fernvale street,condominium,condominium,ocr,strata,2+1,1.0,28,north-east region,sengkang,fernvale,1.398588,103.874632,leasehold/99 years,2021.0,735.0,581.0,u

In [ ]:
df_train.to_csv("train_augmented.csv")
df_test.to_csv("test_augmented.csv")